In [ ]:
import ijson
import sqlite3
from app.s3 import S3
import os 
import shutil

s3 = S3("hydrahi4ai")

In [ ]:
# s3 data path
json_path =  ["data/json/New_collector_20231214_154733.success.json","data/json/Myntra__Marianfer_Cruz_20231219_195028.success.json"]
# s3 img path
s3_img_path = ["ajio-myntra/origin/20231214/", "ajio-myntra/alternative/20231219/"]

# filename db and images
db_name = "trash/db/db_filename.sqlite"
filename_images = "trash/fastdup/path_images.txt"

path_images = "trash/img"

table_name = ["origin", "alternative"]

query_table = "CREATE TABLE IF NOT EXISTS {0} (file_name VARCHAR(1024), ref VARCHAR(256))"
query_insert = "INSERT INTO {0} (file_name, ref) VALUES (?, ?)"

field_name_file_images = ["product_images", "product_images"]
field_name_ref = ["sku", "sku"]

img_per_object = 1

In [ ]:
# creamos la db 
with sqlite3.connect(db_name) as con:
    
    # creamos el archivo donde vamos a guardar la direccion de las imagenes 
    with open(filename_images, "w", encoding="utf-8") as file:

        for i, path in enumerate(json_path):
            
            with open(path, "r", encoding="utf-8") as json:
                
                con.execute(query_table.format(table_name[i]))
                objets_json = ijson.items(json, "item")
                
                for obj in objets_json:
                    
                    images_name = obj[field_name_file_images[i]]
                    ref = obj[field_name_ref[i]]
                    amount = img_per_object if len(images_name) >= img_per_object else len(images_name)
                    
                    for image in images_name[:amount]:
                        
                        if image:
                        
                            path_s3_img = os.path.join(s3_img_path[i], image)
                            con.execute(query_insert.format(table_name[i]), (image, ref))
                            print(path_s3_img)
                            path_local_img = s3.download_file(path_images, path_s3_img)
                            file.write(path_local_img+"\n")
        
        con.commit()
